In [1]:
import numpy as np
import pandas as pd
import h5py
import gc

import sklearn as sk

from sklearn.model_selection import ParameterGrid




import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.utils import GeneratorEnqueuer

import os
base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3'
os.chdir(base_dir)

from Classes.Modeling.DynamicModels import DynamicModels
from Classes.Modeling.StaticModels import StaticModels
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.RamLoader import RamLoader
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.DataProcessing.ts_RamGenerator import data_generator


import sys


import random
import pprint
import re
import json


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3090, compute capability 8.6


In [2]:
class TrainSingleModel(GridSearchResultProcessor):
    
    def __init__(self, x_train, y_train, x_val, y_val, x_test, y_test, noiseAug, helper, loadData, 
                 model_type, num_channels, use_tensorboard, use_liveplots, use_custom_callback,
                 use_early_stopping, use_reduced_lr, log_data = True, results_df = None, 
                 results_file_name = None, index = None):
        self.x_train = x_train
        self.y_train = y_train
        self.x_val = x_val
        self.y_val = y_val
        self.x_test = x_test
        self.y_test = y_test
        
        self.num_channels = num_channels
        
        self.noiseAug = noiseAug
        self.helper = helper
        self.loadData = loadData
        
        self.use_tensorboard = use_tensorboard
        self.use_liveplots = use_liveplots
        self.use_custom_callback = use_custom_callback
        self.use_early_stopping = use_early_stopping
        self.use_reduced_lr = use_reduced_lr
        
        self.num_classes = len(set(self.loadData.label_dict.values()))
        
        self.model_type = model_type
        self.results_df = results_df
        self.log_data = log_data
        
        self.index = index
        
    def create_and_compile_model(self, **p):
        gc.collect()
        tf.keras.backend.clear_session()
        tf.config.optimizer.set_jit(True)
        mixed_precision.set_global_policy('mixed_float16')
        
        epoch = p["epochs"]
        batch_size = p["batch_size"]
        opt = self.helper.get_optimizer(p["optimizer"], p["learning_rate"])
        
        p = self.handle_hyperparams(**p)
        
        if self.index != None:
            model_info = {"model_type" : self.model_type, "index" : self.index}
        else:
            model_info = {"model_type" : self.model_type}
        current_picks = [model_info, p]
        pp = pprint.PrettyPrinter(indent=4)
        pp.pprint(current_picks)
        if self.log_data and self.results_df != None and self.results_file_name != None:
            self.results_df = self.store_params_before_fit_opti(p, self.results_df, self.results_file_name)
        
        _,_,timesteps = self.x_train.shape
        input_shape = (timesteps, self.num_channels)
        
        model = DynamicModels(self.model_type, self.num_classes, input_shape, **p).model
        model_compile_args = self.helper.generate_model_compile_args(opt, self.num_classes)
        model.compile(**model_compile_args)
        return model
        
        
    
    def handle_hyperparams(self, **p):
        if "decay_sequence" or "growth_sequence" in p:
            if "num_filters" in p:
                units_or_num_filters = p["num_filters"]
            else:
                units_or_num_filters = p["units"]
            num_layers = p["num_layers"]
            if "decay_sequence" in p:
                p["decay_sequence"] = self.helper.get_max_decay_sequence(num_layers,
                                                                         units_or_num_filters,
                                                                         p["decay_sequence"],
                                                                         self.num_classes)
            else:
                p["growth_sequence"] = p["growth_sequence"][:num_layers]
        return p
    
    def create_enqueuer(self, X, y, batch_size, noiseAug, num_channels):
        enq = GeneratorEnqueuer(data_generator(X, y, batch_size, noiseAug, num_channels = num_channels, is_lstm  = True), 
                                use_multiprocessing = False)
        return enq
        
    def fit_model(self, model, workers, max_queue_size, **p):
        train_enq = self.create_enqueuer(self.x_train, self.y_train, p["batch_size"], self.noiseAug, self.num_channels)
        val_enq = self.create_enqueuer(self.x_val, self.y_val, p["batch_size"], self.noiseAug, self.num_channels)
        train_enq.start(workers = workers, max_queue_size = max_queue_size)
        val_enq.start(workers = workers, max_queue_size = max_queue_size)
        train_gen = train_enq.get()
        val_gen = val_enq.get()
        
        fit_args = self.helper.generate_fit_args(self.loadData.train, self.loadData.val, self.loadData,
                                                 p["batch_size"], p["epochs"], val_gen,
                                                 use_tensorboard = self.use_tensorboard, 
                                                 use_liveplots = self.use_liveplots, 
                                                 use_custom_callback = self.use_custom_callback,
                                                 use_early_stopping = self.use_early_stopping,
                                                 use_reduced_lr = self.use_reduced_lr)
        try:
            print(f"Utilizes {self.helper.get_steps_per_epoch(self.loadData.val, p['batch_size'])*p['batch_size']}/{len(self.loadData.val)} validation points")
            print(f"Utilizes {self.helper.get_steps_per_epoch(self.loadData.train, p['batch_size'])*p['batch_size']}/{len(self.loadData.train)} training points")
            print("---------------------------------------------------------------------------------")

            # Fit the model using the generated args
            model.fit(train_gen, **fit_args)
            train_enq.stop()
            val_enq.stop()
            del train_gen, val_gen, train_enq, val_enq

        except Exception as e:
            print(str(e))
            print("Something went wrong.")
        return model
    
    def run(self, workers, max_queue_size, **p):
        model = self.create_and_compile_model(**p)
        model = self.fit_model(model, 16, 15, **p)
        self.model = model
        return model

        
            
        
       

In [3]:
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'noise_not_noise' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 1,
    'seed' : 1,
    'subsample_size' : 0.25,
    'balance_non_train_set' : True,
    'use_true_test_set' : False,
    'even_balance' : True
}
loadData = LoadData(**load_args)
train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)
helper = HelperFunctions()

2 2
2 2
2 2
Mapping train redundancy: : [--------------------------------------->] 100 %

Mapping validation redundancy: : [--------------------------------------->] 100 %

Mapping test redundancy: : [--------------------------------------->] 100 %



Loaded explosion and earthquake dataset:
Evenly balanced among classes in the train set.
As well as non train sets.
Distribution (Label: (counts, proportion)) of
Train ds:
earthquake: (20569, 0.5002)  |  explosion: (20554, 0.4998)  
Val ds:
earthquake: (3094, 0.5016)  |  explosion: (3074, 0.4984)  
Test ds:
earthquake: (2069, 0.5032)  |  explosion: (2043, 0.4968)  


In [4]:
use_time_augmentor = True
use_noise_augmentor = True
scaler_name = "normalize"
filter_name = None
band_min = 2
band_max = 4
highpass_freq = 5

use_tensorboard = True
use_liveplots = False
use_custom_callback = True
use_early_stopping = True
start_from_scratch = False
use_reduced_lr = True
log_data = False

load_test_set = True


ramLoader = RamLoader(loadData, 
                      handler, 
                      use_time_augmentor = use_time_augmentor, 
                      use_noise_augmentor = use_noise_augmentor, 
                      scaler_name = scaler_name,
                      filter_name = filter_name, 
                      band_min = band_min,
                      band_max = band_max,
                      highpass_freq = highpass_freq, 
                      load_test_set = load_test_set)
x_train, y_train, x_val, y_val, x_test, y_test, noiseAug = ramLoader.load_to_ram()


Fitting noise set time augmentor: [--------------------------------------->] 100 %

Fit process completed after 9.679181337356567 seconds. Total datapoints fitted: 21199.
Average time per datapoint: 0.00045658669453071217


Fitting train time augmentor: [--------------------------------------->] 100 %

Fit process completed after 13.603031396865845 seconds. Total datapoints fitted: 41123.
Average time per datapoint: 0.0003307888869213298


Fitting validation time augmentor: [--------------------------------------->] 100 %

Fit process completed after 2.0126843452453613 seconds. Total datapoints fitted: 6168.
Average time per datapoint: 0.0003263106915118939


Fitting test time augmentor: [--------------------------------------->] 100 %

Fit process completed after 13.687296390533447 seconds. Total datapoints fitted: 4112.
Average time per datapoint: 0.0033286226630674727


Stage one loading training set, timeAug: [--------------------------------------->] 100 %

Stage one loading valid

In [ ]:
model_type = "CNN_grow"
num_channels = 3

singleModel = TrainSingleModel(x_train, y_train, x_val, y_val, x_test, y_test, 
                               noiseAug, helper, loadData, 
                               model_type, 
                               num_channels, 
                               use_tensorboard, 
                               use_liveplots, 
                               use_custom_callback,
                               use_early_stopping, 
                               use_reduced_lr, 
                               log_data = log_data, 
                               results_df = None, 
                               results_file_name = None, 
                               index = None)

workers = 16
max_queue_size = 15


hyper_params = {
    "num_layers" : 2,
    "batch_size" : 64,
    "epochs" : 25,
    "learning_rate" : 0.01,
    "optimizer" : "sgd",
    "num_filters" : 68,
    "filter_size" : 42,
    "cnn_activation" : "relu",
    "dense_activation" : "relu",
    "padding" : "same",
    "use_layerwise_dropout_batchnorm" : True,
    "growth_sequence" : [1,2,4,4,2,1],
    "dropout_rate" : 0.001,
    "l2_r" : 0.01,
    "l1_r" : 0.001,
    "first_dense_units" : 286,
    "output_layer_activation" : "sigmoid"
}
model = singleModel.run(workers = workers, max_queue_size = max_queue_size, **hyper_params)

[   {'model_type': 'CNN_grow'},
    {   'batch_size': 64,
        'cnn_activation': 'relu',
        'dense_activation': 'relu',
        'dropout_rate': 0.001,
        'epochs': 25,
        'filter_size': 42,
        'first_dense_units': 286,
        'growth_sequence': [1, 2],
        'l1_r': 0.001,
        'l2_r': 0.01,
        'learning_rate': 0.01,
        'num_filters': 68,
        'num_layers': 2,
        'optimizer': 'sgd',
        'output_layer_activation': 'sigmoid',
        'padding': 'same',
        'use_layerwise_dropout_batchnorm': True}]
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 6000, 3)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 6000, 68)          8636      
_________________________________________________________________
batch_normalization (BatchNo (Non